In [328]:
from scipy.spatial import distance as dist
import numpy as np

In [329]:
from imutils.perspective import four_point_transform
import imutils
import cv2
import re

In [330]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized

In [331]:
orig = cv2.imread('mask_out.jpg')
orig1 = cv2.imread("11/5.jpg")
image = orig.copy()
image1 = orig1.copy()
image = resize(image, width=500)
image1 = resize(image1, width=500)
cv2.imwrite('final_out1.jpg', image)
cv2.imwrite('final_out2.jpg', image1)
ratio = orig.shape[1] / float(image.shape[1])

In [332]:
print(ratio)

6.048


In [318]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5,), 0)
edged = cv2.Canny(blurred, 75, 200)

In [319]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [320]:
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

In [321]:
cnts = grab_contours(cnts)

In [322]:
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [323]:
print(cnts)

[array([[[185,  57]],

       [[183,  59]],

       [[183,  60]],

       [[182,  61]],

       [[182,  64]],

       [[181,  65]],

       [[181,  75]],

       [[180,  76]],

       [[180,  87]],

       [[179,  88]],

       [[179,  93]],

       [[178,  94]],

       [[178,  98]],

       [[177,  99]],

       [[177, 102]],

       [[176, 103]],

       [[176, 105]],

       [[175, 106]],

       [[175, 110]],

       [[174, 111]],

       [[174, 116]],

       [[173, 117]],

       [[173, 141]],

       [[172, 142]],

       [[172, 153]],

       [[171, 154]],

       [[171, 158]],

       [[170, 159]],

       [[170, 160]],

       [[169, 161]],

       [[169, 162]],

       [[168, 163]],

       [[168, 165]],

       [[167, 166]],

       [[167, 168]],

       [[166, 169]],

       [[166, 172]],

       [[165, 173]],

       [[165, 247]],

       [[164, 248]],

       [[164, 260]],

       [[163, 261]],

       [[163, 265]],

       [[162, 266]],

       [[162, 267]],

       [[

In [324]:
receiptCnt = None
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    # if our approximated contour has four points, then we can
    # assume we have found the outline of the receipt
    if len(approx) == 4:
        receiptCnt = approx
        break
# if the receipt contour is empty then our script could not find the
# outline and we should be notified
print(receiptCnt)
if receiptCnt is None:
    raise Exception(("Could not find receipt outline. "
        "Try debugging your edge detection and contour steps."))

[[[185  57]]

 [[128 532]]

 [[309 560]]

 [[352 100]]]


In [325]:
def order_points(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [326]:
receipt = four_point_transform(orig1, receiptCnt.reshape(4, 2) * ratio)

In [327]:
cv2.imwrite('final_out.jpg', resize(receipt, width=500))

True

In [ ]:
#IMG_3588 5.18.41 AM.jpg
#IMG_3609 5.18.41 AM.jpg
#IMG_3610 5.18.41 AM.jpg